<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/lightgbm_enhanced_strategy_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y
!apt-get clean

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cuda-toolkit-12-4-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-7' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-8' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-5-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-2' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-3' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-4' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-5' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-6' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-8' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-9' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-6-config

In [2]:
#Protocol Buffer Fix (for TensorFlow)
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


In [3]:
#Update Colab Environment and System Libraries
!apt-get update -y && apt-get upgrade -y


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,750 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,036 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,347 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy

In [4]:
#Install Correct Version of CUDA for Colab GPU
!apt-get update -qq && apt-get install -y \
    libcusolver11 libcusparse11 libcurand10 libcufft10 libnppig10 libnppc10 libnppial10 \
    cuda-toolkit-12-4

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libnppig10
E: Unable to locate package libnppc10
E: Unable to locate package libnppial10


In [5]:
#Set Correct CUDA Paths
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'
os.environ['PATH'] += ':/usr/local/cuda-12.4/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.4/lib64'


In [6]:
#Install RAPIDS and NVIDIA Dependencies
!pip install --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==25.2.0 cudf-cu12==25.2.0 cupy-cuda12x dask-cuda==25.2.0 dask-cudf-cu12==25.2.0


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 66.7 MB/s eta 0:00:00
  Attempting uninstall: cudf-cu12
    Found existing installation: cudf-cu12 25.2.1
    Uninstalling cudf-cu12-25.2.1:
      Successfully uninstalled cudf-cu12-25.2.1
  Attempting uninstall: dask-cudf-cu12
    Found existing installation: dask-cudf-cu12 25.2.2
    Uninstalling dask-cudf-cu12-25.2.2:
      Successfully uninstalled dask-cudf-cu12-25.2.2
  Attempting uninstall: cuml-cu12
    Found existing installation: cuml-cu12 25.2.1
    Uninstalling cuml-cu12-25.2.1:
      Successfully uninstalled cuml-cu12-25.2.1


In [7]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0

#Install Stable Baselines3 and Trading Libraries
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib

#Install Miscellaneous Libraries
!pip install matplotlib scikit-learn pandas numba==0.61.0

#Install PyTorch with GPU Support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [8]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0


import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow GPU memory growth enabled")
    except RuntimeError as e:
        print(f"TensorFlow GPU memory config failed: {e}")


TensorFlow GPU memory growth enabled


In [9]:
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance --quiet
!pip install stable-baselines3[extra] --quiet


In [10]:
!rm -rf /content/drive

In [12]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [13]:
#Add Live Updating / Online Learning
!pip install yfinance lightgbm scikit-learn matplotlib


In [14]:
#Noise Filtering
!pip install PyWavelets

In [30]:
pip install -U scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 67.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.0 which is incompatible.


In [8]:
# === Imports and Warning Fixes ===
import os, gc, time, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pywt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from lightgbm import LGBMClassifier, early_stopping, log_evaluation

# Fix FutureWarning for 'ensure_all_finite'
warnings.filterwarnings("ignore", message=".*ensure_all_finite.*", category=FutureWarning)

# === Config ===
TEST_MODE = False
TICKERS = ['AAPL'] if TEST_MODE else [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO', 'BAC', 'ABBV',
    'AVGO', 'COST', 'DIS', 'WMT', 'PFE', 'CSCO', 'TMO', 'ACN', 'CVX', 'ABT', 'CRM',
    'MCD', 'INTC', 'DHR', 'QCOM', 'TXN', 'NEE', 'AMGN', 'LIN', 'PM', 'BMY', 'MDT',
    'HON', 'UNP', 'LOW', 'ORCL', 'NKE', 'RTX', 'MS', 'GS'
]

PERIOD = "720d"
INTERVAL = "1h"
SAVE_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm/lightgbm_walkforward_models"
RESULTS_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm"
OUTPUT_PATH = f"{RESULTS_DIR}/multi_stock_feature_engineered_dataset.csv"
SLIPPAGE_RATE = 0.001
THRESHOLD = 0.55
STOP_LOSS_PCT = 0.03
FEATURES = ["SMA_50", "EMA_20", "RSI_voladj", "MACD_voladj", "Signal_Line", "ATR_voladj", "OBV", "CCI_voladj"]

os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "plots"), exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "data"), exist_ok=True)

# === Noise Filtering ===
def wavelet_denoise(series, wavelet="db1"):
    coeffs = pywt.wavedec(series, wavelet, mode="symmetric")
    coeffs[1:] = [np.zeros_like(c) for c in coeffs[1:]]
    return pywt.waverec(coeffs, wavelet, mode="symmetric")[:len(series)]

# === Market Regime Detection ===
def detect_regime(df, window=100):
    df["Return"] = df["Close"].pct_change()
    df["Vol"] = df["Close"].rolling(window).std()
    valid = df[["Return", "Vol"]].dropna()
    if len(valid) < 50:
        df["Regime"] = 1
        return df
    model = KMeans(n_clusters=3, random_state=42)
    labels = model.fit_predict(valid)
    regime = pd.Series(index=valid.index, data=labels)
    df["Regime"] = regime.reindex(df.index).ffill().bfill()
    return df

# === Feature Engineering ===
def compute_features(df):
    df['SMA_50'] = df['Close'].rolling(50).mean()
    df['EMA_20'] = df['Close'].ewm(span=20).mean()
    delta = df['Close'].diff()
    gain = delta.clip(lower=0).rolling(14).mean()
    loss = -delta.clip(upper=0).rolling(14).mean()
    rs = gain / (loss + 1e-6)
    df['RSI'] = 100 - (100 / (1 + rs))
    df['MACD'] = df['Close'].ewm(12).mean() - df['Close'].ewm(26).mean()
    df['Signal_Line'] = df['MACD'].ewm(9).mean()
    df['ATR'] = df['High'].rolling(14).max() - df['Low'].rolling(14).min()
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (tp - tp.rolling(20).mean()) / (0.015 * tp.rolling(20).std())
    for col in ["RSI", "MACD", "CCI", "OBV", "ATR"]:
        df[f"{col}_voladj"] = wavelet_denoise(df[col].fillna(0).values)
    df = detect_regime(df)
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    return df

# === Robust Downloader ===
def download_data(ticker, retries=3):
    for attempt in range(retries):
        try:
            df = yf.download(ticker, period=PERIOD, interval=INTERVAL, progress=False)
            if not df.empty:
                df.reset_index(inplace=True)
                df['Symbol'] = ticker
                return df
        except Exception as e:
            print(f"Retry {attempt+1} for {ticker}: {e}")
            time.sleep((attempt + 1) * 5)
    print(f"Failed: {ticker}")
    return None

# === Save Preprocessed Dataset ===
all_data = []
for ticker in TICKERS:
    print(f"Processing {ticker}")
    df_raw = download_data(ticker)
    if df_raw is None:
        continue
    try:
        df_feat = compute_features(df_raw)
        all_data.append(df_feat)
    except Exception as e:
        print(f"Feature failure: {ticker} -> {e}")

if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    final_df.to_csv(OUTPUT_PATH, index=False)
    print(f"Saved to {OUTPUT_PATH} | Shape: {final_df.shape}")
else:
    print("No data processed")

# === Walkforward LGBM Training & Evaluation ===
def run_lgbm_walkforward(ticker, window_size=2000, step_size=250, initial_cash=100000):
    print(f"\n{ticker} | LGBM Walkforward")
    df = yf.download(ticker, period=PERIOD, interval=INTERVAL, auto_adjust=False, progress=False)
    if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.droplevel(1)
    df = compute_features(df)
    if df.empty or not all(col in df.columns for col in FEATURES): return

    scaler = MinMaxScaler()
    capital = initial_cash
    shares = capital / df['Close'].iloc[0]
    model = LGBMClassifier(n_estimators=100, learning_rate=0.05, random_state=42, force_row_wise=True)
    port_lgbm, port_hold, y_preds, y_true = [], [], [], []

    for start in range(0, len(df) - window_size, step_size):
        segment = df.iloc[start:start + window_size].copy()
        if segment.shape[0] < window_size: break
        X = scaler.fit_transform(segment[FEATURES])
        y = segment['Target']
        X_train, X_test = X[:-step_size], X[-step_size:]
        y_train, y_test = y[:-step_size], y[-step_size:]
        prices = segment['Close'].values[-step_size:]

        model.fit(X_train, y_train,
                  eval_set=[(X_test, y_test)],
                  callbacks=[early_stopping(20), log_evaluation(0)])

        probs = model.predict_proba(X_test)[:, 1]
        preds = (probs > THRESHOLD).astype(int)
        y_preds.extend(preds)
        y_true.extend(y_test.tolist())

        for i, pred in enumerate(preds):
            raw_price = prices[i]
            exec_price = raw_price * (1 + SLIPPAGE_RATE) if pred else raw_price * (1 - SLIPPAGE_RATE)
            if pred:
                if raw_price < exec_price * (1 - STOP_LOSS_PCT):
                    capital = shares * raw_price
                else:
                    capital = shares * exec_price
            else:
                shares = capital / exec_price
            port_lgbm.append(capital)
            port_hold.append(shares * raw_price)
        time.sleep(0.01)

    if len(y_preds) < 2: return
    y_preds, y_true = np.array(y_preds), np.array(y_true[:len(y_preds)])
    returns = np.diff(port_lgbm) / (np.array(port_lgbm[:-1]) + 1e-6)
    sharpe = np.mean(returns) / (np.std(returns) + 1e-6) * np.sqrt(252)
    drawdown = np.max(np.maximum.accumulate(port_lgbm) - port_lgbm)

    metrics = {
        "Ticker": ticker,
        "Final_Portfolio": round(port_lgbm[-1], 2),
        "Return_%": round((port_lgbm[-1] - initial_cash) / initial_cash * 100, 2),
        "Sharpe": round(sharpe, 4),
        "Accuracy": round(accuracy_score(y_true, y_preds), 4),
        "Precision": round(precision_score(y_true, y_preds), 4),
        "Recall": round(recall_score(y_true, y_preds), 4),
        "F1_Score": round(f1_score(y_true, y_preds), 4),
        "Drawdown": round(drawdown, 2)
    }

    pd.DataFrame([metrics]).to_csv(f"{RESULTS_DIR}/metrics_{ticker}.csv", index=False)
    model.booster_.save_model(f"{SAVE_DIR}/model_{ticker}.txt")

    plt.figure(figsize=(12, 6))
    plt.plot(port_lgbm, label="LGBM Strategy")
    plt.plot(port_hold, label="Buy & Hold")
    plt.title(f"{ticker} Portfolio Value")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/plots/{ticker}_portfolio_plot.png")
    plt.close()
    print(f"{ticker}: Return {metrics['Return_%']}%, Sharpe {metrics['Sharpe']}")

    summary_path = os.path.join(RESULTS_DIR, "lightgbm_walkforward_summary.csv")
    if os.path.exists(summary_path):
        all_df = pd.read_csv(summary_path)
        all_df = all_df[all_df["Ticker"] != ticker]
        all_df = pd.concat([all_df, pd.DataFrame([metrics])], ignore_index=True)
    else:
        all_df = pd.DataFrame([metrics])
    all_df.to_csv(summary_path, index=False)

# === Run All Tickers ===
for ticker in TICKERS:
    run_lgbm_walkforward(ticker)
    gc.collect()
    time.sleep(1)


Processing AAPL


/tmp/ipython-input-8-3001232173.py:87: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=PERIOD, interval=INTERVAL, progress=False)


Processing TSLA


/tmp/ipython-input-8-3001232173.py:87: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=PERIOD, interval=INTERVAL, progress=False)


Processing MSFT


...

Saved to /content/drive/MyDrive/QuantConnect/results_lightgbm/multi_stock_feature_engineered_dataset.csv | Shape: (255683, 279)

AAPL | LGBM Walkforward
[LightGBM] [Info] Number of positive: 894, number of negative: 856
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.510857 -> initscore=0.043435
[LightGBM] [Info] Start training from score 0.043435
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[3]	valid_0's binary_logloss: 0.691331
[LightGBM] [Info] Number of positive: 898, number of negative: 852
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.513143 -> initscore=0.052584
[LightGBM] [Info] Start training from score 0.052584
Training until validation scores don't improve for 20 rounds
Ear

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


MRK: Return 0.0%, Sharpe 0.0

PEP | LGBM Walkforward
[LightGBM] [Info] Number of positive: 900, number of negative: 850
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.514286 -> initscore=0.057158
[LightGBM] [Info] Start training from score 0.057158
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.694878
[LightGBM] [Info] Number of positive: 901, number of negative: 849
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.514857 -> initscore=0.059446
[LightGBM] [Info] Start training from score 0.059446
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[39]	valid_0's binary_logloss: 0.688585
[LightGBM] [Info] Number of 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


RTX: Return 0.0%, Sharpe 0.0

MS | LGBM Walkforward
[LightGBM] [Info] Number of positive: 871, number of negative: 879
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497714 -> initscore=-0.009143
[LightGBM] [Info] Start training from score -0.009143
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[2]	valid_0's binary_logloss: 0.692178
[LightGBM] [Info] Number of positive: 855, number of negative: 895
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.488571 -> initscore=-0.045722
[LightGBM] [Info] Start training from score -0.045722
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[13]	valid_0's binary_logloss: 0.690796
[LightGBM] [Info] Number 